This code takes positional data, an experimental schedule, and a video. It combines the first two and uses the third to interpose a display on the image. 

In [9]:
import cmath
import math
import cv2 as cv
import pandas as pd
import numpy as np
import simba.mixins.circular_statistics as si


In [1]:
def encoding(enumb, data, onea):
    """helper function, please ignore"""
    current = onea[onea["Evnt_ID"] == enumb]
    times = set(current["Evnt_Time"]) #was set
    if enumb == 39:
        data.loc[data["Time"].isin(times), str(enumb)] = -1
    else:
        data.loc[data["Time"].isin(times), str(enumb)] = 1

def append(sched, posdata, time=0, framerate = 20):
    """Takes a csv containing an experiental schedule, positional data of the mouse, any time offset from starting, and combines them based off time."""   
    onea = pd.read_csv(sched)

    onea = onea[onea["Evnt_Time"] != 0]
    pdata = np.load(posdata)

    data = pdata.reshape(pdata.shape[0], -1)
    data = pd.DataFrame(data)

    data = data.rename(columns={0: "NOSE X", 1: "NOSE Y", 2: "NOSE P",
                                3: "lEAR X", 4: "lEAR Y", 5: "lEAR P",
                                6: "rEAR X", 7: "rEAR Y", 8: "rEAR P",
                                9: "lBODY X", 10: "lBODY Y", 11: "lBODY P",
                                12: "rBODY X", 13: "rBODY Y", 14: "rBODY P",
                                15: "CENTER X", 16: "CENTER Y", 17: "CENTER P",
                                18: "TAIL X", 19: "TAIL Y", 20: "TAIL P"})

    data["Time"] = data.index / framerate
    filler = np.zeros(len(data))
    new_col = {
        "1": filler,
        "7": filler,
        "16": filler,
        "21": filler,
        "29": filler,
        "38": filler,
        "39": filler
    }

    data = data.assign(**new_col)
    onea["Evnt_Time"] = round(onea["Evnt_Time"] * 20 ) / 20

    for i in new_col.keys():
        encoding(int(i), data, onea)
    data = data[data["Time"] >= time]
    data["Time"] = data["Time"] - time
    return data

In [4]:
schedname = "3A_Touch_5CSRT_MouseTraining1_s2_a1.csv" #this is a dummy bar, we don't have schedule data for this one yet
posname = r"data\2C_Mouse_5-choice_MustTouchTraining_a1_greyscale.npy"
vname = r'C:\Users\naimc\Desktop\work\final\data\2C_Mouse_5-Choice_MustTouchTraining_a1_greyscale.mp4'
timestart = 0 # round to nearest twentieth

In [12]:
fin = append(schedname, posname, timestart)


In [13]:
def fixit(colname, df):
    return scipy.signal.savgol_filter(
                    x=df[colname].to_numpy(),
                    window_length=3,
                    polyorder=2,
                    mode="nearest",)

fin["NOSE X"] = fixit("NOSE X", fin)
fin["NOSE Y"] = fixit("NOSE Y", fin)
fin["rEAR X"] = fixit("rEAR X", fin)
fin["rEAR Y"] = fixit("rEAR Y", fin)
fin["lEAR X"] = fixit("lEAR X", fin)
fin["lEAR Y"] = fixit("lEAR Y", fin)

fin["direction"] = si.CircularStatisticsMixin.direction_three_bps(nose_loc=np.column_stack((fin["NOSE Y"], fin["NOSE X"])).astype(np.float32),
                                                                  left_ear_loc=np.column_stack((fin["lEAR Y"], fin["lEAR X"])).astype(np.float32), 
                                                                  right_ear_loc=np.column_stack((fin["rEAR Y"], fin["rEAR X"])).astype(np.float32)).astype(np.float32)
fin["direction"] = (180-fin["direction"]) % 360 #CHECK IF IT'S DUE 

fin["Circular Mean"] = si.CircularStatisticsMixin.sliding_circular_mean(data=np.array(fin["direction"]), time_windows=np.array([0.8]), fps=20)
fin.loc[fin.index < 10, "Circular Mean"] = np.radians(fin.loc[fin.index< 10, "direction"])
fin["Slope"] = np.tan((np.asarray(fin["Circular Mean"])))
fin["Circular Mean"] = np.degrees(np.asarray(fin["Circular Mean"]))
fin.loc[fin.index< 4, "Circular Mean"] = fin.loc[fin.index< 4, "direction"]

In [ ]:
wtarget = 200
midpoint = 360
frameheight = 720

def slopechecker(row, midpoint, wtarget):
    # print(row["Circular Mean"])
    if row["Circular Mean"] > 180:
        return 0
    if row["Circular Mean"] == 90 and row["NOSE X"] - midpoint < wtarget/2:
        return 1
    disp = (frameheight - row["NOSE Y"])/row["Slope"]+row["NOSE X"]
    # print(disp - midpoint)
    if abs(disp - midpoint) < wtarget/2:
        return 1
    return 0


"""
This is an internet function because i don't get complex numbers very well
"""

def average_angle_degrees(angles_degrees):
    angles_radians = np.radians(angles_degrees)
    complex_numbers = np.exp(1j * angles_radians)
    
    mean_complex = np.mean(complex_numbers)
    mean_angle_radians = cmath.phase(mean_complex)
    
    # Convert the result back to degrees
    mean_angle_degrees = np.degrees(mean_angle_radians)
    
    # Ensure the result is in the range [0, 360)
    mean_angle_degrees = (mean_angle_degrees + 360) % 360
    
    return mean_angle_degrees

In [ ]:
wtarget = 200
midpoint = 360
def selection(df, index, range, midpoint, wtarget):
    # ans = np.zeros(range)
    current = df.iloc[index:(index+range)]#[["Circular Mean", "NOSE Y", "Slope", "NOSE X"]]
    meanDirection = si.CircularStatisticsMixin.direction_three_bps(nose_loc=np.column_stack((current["NOSE X"].mean(), current["NOSE Y"].mean())).astype(np.float32),
                                                                 left_ear_loc=np.column_stack((current["lEAR Y"].mean(), current["lEAR X"].mean())).astype(np.float32), 
                                                                 right_ear_loc=np.column_stack((current["rEAR Y"].mean(), current["rEAR X"].mean())).astype(np.float32)).astype(np.float32)*2
    # PLACEHOLDER APPlY
    # if "view" in current.columns:
    #     current.drop("view", axis=1)
    times = current.apply(slopechecker, axis=1)
    current["view"] = times
    return [current["view"], meanDirection]

fin["dist"] = np.sqrt(((fin["NOSE X"]-680))**2+(fin["NOSE Y"])**2)

In [16]:
# vname = r'C:\Users\naimc\Desktop\work\final\data\2C_Mouse_5-choice_MustTouchTraining_a1_greyscale.mp4'
video_capture = cv.VideoCapture(vname)  
outname = "2C.mp4"

fps = int(video_capture.get(cv.CAP_PROP_FPS))
width = int(video_capture.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(video_capture.get(cv.CAP_PROP_FRAME_HEIGHT))

thresh = 0.4

fourcc = cv.VideoWriter_fourcc(*'mp4v')
output_video = cv.VideoWriter(outname, fourcc, fps, (width, height), isColor=True)


count = 0
bad = []
for i in fin["direction"]:
    if i > 180:
        bad.append(360-i)
    elif i < 0:
        bad.append(i*-1)
    else:
        bad.append(i)
fin["dist"] = np.sqrt((fin["NOSE X"]-680)**2+(fin["NOSE Y"])**2)
while True:
    
    # print(count)
    ret, frame = video_capture.read()
    
    # Break the loop if we have reached the end of the video
    if not ret:
        break
    
    # Convert the frame to grayscale
    img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY) # was grey frame
    img = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
    
    # img = cv.putText(img, text=str(bad[count]), org=(120, 120), fontFace = cv.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 255), thickness=2)
    
    if fin.loc[count]["dist"] < 100:
        img = cv.putText(img, text="On Target", org=(120, 120), fontFace = cv.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 255), thickness=2)

    # # Write the grayscale frame to the output video
    # output_video.write(gray_frame)
    if fin.loc[count]["NOSE P"] > thresh:
        img = cv.circle(img, (int(fin.loc[count]["rEAR X"]), int(fin.loc[count]["rEAR Y"])), radius=3, color=(255, 0, 0), thickness=-1)
    if fin.loc[count]["lEAR P"] > thresh:
        img = cv.circle(img, (int(fin.loc[count]["lEAR X"]), int(fin.loc[count]["lEAR Y"])), radius=3, color=(0, 255, 0), thickness=-1)
    if fin.loc[count]["lBODY P"] > thresh:
        img = cv.circle(img, (int(fin.loc[count]["lBODY X"]), int(fin.loc[count]["lBODY Y"])), radius=3, color=(255, 255, 0), thickness=-1)
    if fin.loc[count]["rBODY P"] > thresh:
        img = cv.circle(img, (int(fin.loc[count]["rBODY X"]), int(fin.loc[count]["rBODY Y"])), radius=3, color=(0, 255, 255), thickness=-1)
    if fin.loc[count]["TAIL P"] > thresh:
        img = cv.circle(img, (int(fin.loc[count]["TAIL X"]), int(fin.loc[count]["TAIL Y"])), radius=3, color=(255, 0, 255), thickness=-1)
    if fin.loc[count]["CENTER P"] > thresh:
        img = cv.circle(img, (int(fin.loc[count]["CENTER X"]), int(fin.loc[count]["CENTER Y"])), radius=3, color=(0, 0, 0), thickness=-1)

    point1x = fin.loc[count]["NOSE X"]
    point1y = fin.loc[count]["NOSE Y"]
    dir = fin.loc[count]["direction"] # or C
    #issue?
    point2x = point1x + 100*math.cos(np.radians(dir))
    point2y = point1y + 100*math.sin(np.radians(dir))
    roverr = (point2y-point1y)/(point2x-point1x)
    # cv.circle(img, (1280, 720), radius = 20, color=(0, 0, 255), thickness=-1)
    # THIS IS WRONG FIX TOMORROW MORNING - sope going to the right is negative
    if fin.loc[count]["NOSE P"] > thresh and fin.loc[count]["lEAR P"] > thresh and fin.loc[count]["rEAR P"] > thresh:
        if dir < 180: #change in x times the slope is the rise, plus starting rise.
            #y divided by the slope is the run
            # change
            outornot = (720-fin.loc[count]["NOSE Y"])/roverr+fin.loc[count]["NOSE X"]
            # outornot = -(fin.loc[count]["NOSE Y"])/roverr+fin.loc[count]["NOSE X"]
            # img = cv.putText(img, text=str(roverr), org=(120, 120), fontFace = cv.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 255), thickness=2)
            # print(outornot)
            sope = (720-fin.loc[count]["NOSE Y"])/(outornot-fin.loc[count]["NOSE X"])
            if outornot > 1280:
                cv.circle(img, (1278, int(720-sope*(outornot-1280))), radius=3, color=(0, 0, 255), thickness=-1)
                # cv.circle(img, (1278, int(720-roverr*(fin.loc[count]["NOSE X"]-1280))), radius=3, color=(0, 0, 255), thickness=-1)
            elif outornot < 0:
                cv.circle(img, (5, int(720-sope*(outornot-0))), radius=3, color=(0, 0, 255), thickness=-1)
            else:
                cv.circle(img, (round(outornot), 720), radius=5, color=(0, 0, 255), thickness=-1)
        else:
            outornot = -(fin.loc[count]["NOSE Y"])/roverr+fin.loc[count]["NOSE X"]
            # outornot = (720-fin.loc[count]["NOSE Y"])/roverr+fin.loc[count]["NOSE X"]
            # img = cv.putText(img, text=(str(roverr)), org=(120, 120), fontFace = cv.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 255), thickness=2)
            # print(outornot)
            sope = (fin.loc[count]["NOSE Y"])/(outornot-fin.loc[count]["NOSE X"])
            if outornot > 1280:
                # cv.circle(img, (1278, int(roverr*(fin.loc[count]["NOSE X"]-1280))), radius=3, color=(0, 0, 255), thickness=-1)
                cv.circle(img, (1278, int(sope*(outornot-1280))), radius=3, color=(0, 0, 255), thickness=-1)
            elif outornot < 0:
                cv.circle(img, (5, int(sope*(outornot-0))), radius=3, color=(0, 0, 255), thickness=-1)
            else:
                cv.circle(img, (round(outornot), 0), radius=5, color=(0, 0, 255), thickness=-1)

        img = cv.line(img, [int(point1x), int(point1y)], [int(point2x), int(point2y)], color=(0, 0, 255), thickness=2)
        img = cv.circle(img, (int(fin.loc[count]["NOSE X"]), int(fin.loc[count]["NOSE Y"])), radius=3, color=(0, 0, 255), thickness=-1)
 

    # Display the grayscale frame (optional)
    output_video.write(img)
    cv.imshow('Grayscale Video', img)
    
    # Break the loop if 'q' is pressed
    count += 1
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    # time.sleep(1/(60))

# Release the video capture and writer objects
video_capture.release()
output_video.release()

# Close any open windows (if applicable)
cv.destroyAllWindows()